In [1]:
$ 저장 

훈련하는 도중이나 훈련이 끝난 후에 모델을 저장할 수 있음 
모델을 중지된 지점부터 다시 훈련하게 할 수 있음 -> 한번에 오랫동안 훈련하지 않아도 됨

* 모델을 만드는 코드 & 모델의 훈련된 가중치 또는 파라미터 -> 해당 데이터의 공유를 통해 다른사람이 모델 참조 가능


SyntaxError: invalid syntax (<ipython-input-1-d15914eb53ba>, line 1)

In [2]:
# 필요한 라이브러리 설치 

!pip install -q h5py pyyaml 

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.13.1'

In [4]:
# 데이터 셋 받기 
# 모델 실행속도를 빠르게 하기 위해 샘플에서 처음 1000개만 사용 


(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [5]:
# 가중치를 저장하고 불러오는 예제를 위해 만든 간단한 모델 


# 간단한 Sequential 모델을 반환합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(), 
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model


# 모델 객체를 만듭니다
model = create_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 훈련하는 동안 체크포인트 저장하기 
# 다시 훈련하지 않고 모델을 재사용하거나 훈련 과정이 중지된 경우 이어서 훈련을 진행 가능 

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # 훈련 단계에 콜백을 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 960/1000 [===========================>..] - ETA: 0s - loss: 1.1707 - acc: 0.6667
Epoch 00001: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1446 - acc: 0.6750 - val_loss: 0.7255 - val_acc: 0.7770
Epoch 2/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.4153 - acc: 0.8823
Epoch 00002: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 1s 711us/sample - loss: 0.4129 - acc: 0.8830 - val_loss: 0.5264 - val_acc: 0.8340
Epoch 3/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.2819 - acc: 0.9284
Epoch 00003: saving model to training_1/cp.ckpt

Consider using a TensorFlow optim

In [7]:
# 텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트한다 

!dir {checkpoint_dir}

 C 드라이브의 볼륨: Windows
 볼륨 일련 번호: EABA-67CB

 C:\Users\bsww201\Documents\tensorflowTest\TensorflowTutorial\training_1 디렉터리

2019-05-04  오전 03:42    <DIR>          .
2019-05-04  오전 03:42    <DIR>          ..
2019-05-04  오전 03:42                71 checkpoint
2019-05-04  오전 03:42         1,631,508 cp.ckpt.data-00000-of-00001
2019-05-04  오전 03:42               648 cp.ckpt.index
               3개 파일           1,632,227 바이트
               2개 디렉터리  162,326,183,936 바이트 남음


In [8]:
# 훈련하지 않은 새로운 모델 만들어 보기 
# 가중치만 복원할 땐 원본 모델과 동일한 구조로 모델을 만들어야 함

# 훈련하지 않은 새 모델을 만들고 테스트 세트에서 평가 -> 운련되지 않은 모델의 성능은 무작위로 선택하는 수준(~10%)
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 172us/sample - loss: 2.4026 - acc: 0.0760
훈련되지 않은 모델의 정확도:  7.60%


In [9]:
# 체크 포인트에서 가중치를 로드하고 다시 평가 

model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 99us/sample - loss: 0.4021 - acc: 0.8660
복원된 모델의 정확도: 86.60%


In [10]:
# 체크 포인트 콜백함수 - ModelCheckpoint
# 이 콜백함수는 몇가지 매개변수 제공 
# 체크포인트 이름을 고유하게 만들거나 체크포인트 주기를 조정할 수 있음 

## 새로운 모델을 훈련하고 다섯번의 에포크마다 고유한 이름으로 체크포인트를 저장해보기 

# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 다섯 번째 에포크마다 가중치를 저장합니다
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00005: saving model to training_2/cp-0005.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00010: saving model to training_2/cp-0010.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00015: saving model to training_2/cp-0015.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00020: saving model to training_2/cp-0020.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00025: saving model to training_2/cp-0025.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00030: saving model to training_2/cp-0030.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00035: saving model to training_2/cp-0035.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00040: saving model to training_2/cp-0040.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00045: saving model to training_2/cp-0045.ckpt

Consider 

In [11]:
! dir {checkpoint_dir}

 C 드라이브의 볼륨: Windows
 볼륨 일련 번호: EABA-67CB

 C:\Users\bsww201\Documents\tensorflowTest\TensorflowTutorial\training_2 디렉터리

2019-05-04  오전 03:42    <DIR>          .
2019-05-04  오전 03:42    <DIR>          ..
2019-05-04  오전 03:42                81 checkpoint
2019-05-04  오전 03:42         1,631,508 cp-0000.ckpt.data-00000-of-00001
2019-05-04  오전 03:42               648 cp-0000.ckpt.index
2019-05-04  오전 03:42         1,631,508 cp-0005.ckpt.data-00000-of-00001
2019-05-04  오전 03:42               648 cp-0005.ckpt.index
2019-05-04  오전 03:42         1,631,508 cp-0010.ckpt.data-00000-of-00001
2019-05-04  오전 03:42               648 cp-0010.ckpt.index
2019-05-04  오전 03:42         1,631,508 cp-0015.ckpt.data-00000-of-00001
2019-05-04  오전 03:42               648 cp-0015.ckpt.index
2019-05-04  오전 03:42         1,631,508 cp-0020.ckpt.data-00000-of-00001
2019-05-04  오전 03:42               648 cp-0020.ckpt.index
2019-05-04  오전 03:42         1,631,508 cp-0025.ckpt.data-00000-of-00001
2019-05-04  오전 03:42   

In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [13]:
# create_model을 이용해 모델 초기화
# 최근 체크포인트를 로드하여 테스트하기 

model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 174us/sample - loss: 0.4929 - acc: 0.8720
복원된 모델의 정확도: 87.20%


In [14]:
$ 체크포인트 파일 

* 위 코드는 가중치를 일련의 체크포인트 포맷의 파일에 저장
* 해당 파일에 포함되는 것은 훈련된 이진포맷의 가중치 

** 체크포인트가 담고있는 것
     - 모델의 가중치를 포함하는 하나 이상의 shard 
     - 가중치가 어드 shard에 저장되어 잇는지를 나타내는 인덱스 파일 
     --> 단일 머신에서 모델을 훈련했다면 -> .data-00000-of-00001 확장자를 가진 샤드 하나만 만들어짐 

SyntaxError: invalid syntax (<ipython-input-14-93135a94f82c>, line 1)

In [15]:
# 가중치 수동으로 저장하기 

## 지금까지 위에서는 가중치를 모델에 로드하는 방법 사용 
## 수동으로 가중치를 저장하기 위해서는 --> Model.save_weights 메서드를 사용

# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 가중치를 복원합니다
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))


Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 187us/sample - loss: 0.4929 - acc: 0.8720
복원된 모델의 정확도: 87.20%


In [16]:
$ 모델 전체를 저장하기 

* 전체모델을 파일 하나에 저장할 수 있다 
* 여기에는 가중치, 모델 구성, 옵티마이저에 지정한 설정까지 포함됨
* 모델의 체크포인트를 저장하므로 원본 코드를 사용하지 않고 나중에 정확이 동일한 상태에서 훈련 다시 시작 가능 

* 사용 되는 방법 
    - Tensorflow.js 로 모델을 로드한 다음 웹 브라우저에서 모델을 훈련하고 실행할 수 있다
    - 모바일 장치에 맞도록 변환한 다음 tensorflow Lite를 사용해 실행 가능
    --> hdf5, saved model

SyntaxError: invalid syntax (<ipython-input-16-143f4e79befc>, line 1)

In [17]:
# hdf5 파일로 저장하기 
## 저장된 모델을 하나의 이진 파일처럼 다룰 수 있음 

model = create_model()

model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 1s 888us/sample - loss: 1.1696 - acc: 0.6620
Epoch 2/5
1000/1000 [==============================] - 1s 517us/sample - loss: 0.4172 - acc: 0.8860
Epoch 3/5
1000/1000 [==============================] - 1s 571us/sample - loss: 0.2775 - acc: 0.9320
Epoch 4/5
1000/1000 [==============================] - 1s 536us/sample - loss: 0.2202 - acc: 0.9440
Epoch 5/5
1000/1000 [==============================] - 1s 540us/sample - loss: 0.1640 - acc: 0.9630


In [18]:
# 위에서 만든 파일을 이용해 다시 모델 만들어 보기 

# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
# 다시 만든 모델의 정확도 확인 

loss, acc = new_model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 192us/sample - loss: 0.4703 - acc: 0.8380
복원된 모델의 정확도: 83.80%


In [20]:
# saved_model 을 사용해 저장 

## 해당 메서드는 실험적이므로 변경될 가능성 있음 


# 새로운 모델 만들기 
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 1s 804us/sample - loss: 1.1554 - acc: 0.6810
Epoch 2/5
1000/1000 [==============================] - 1s 597us/sample - loss: 0.4101 - acc: 0.8880
Epoch 3/5
1000/1000 [==============================] - 1s 560us/sample - loss: 0.2803 - acc: 0.9250
Epoch 4/5
1000/1000 [==============================] - 1s 645us/sample - loss: 0.2026 - acc: 0.9540
Epoch 5/5
1000/1000 [==============================] - 1s 621us/sample - loss: 0.1504 - acc: 0.9650


In [21]:
# saves_model 만들기 

saved_model_path = tf.contrib.saved_model.save_keras_model(model, "./saved_models")


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



NotFoundError: Failed to create a directory: ./saved_models\1556909014; No such file or directory

In [54]:
# 타임스탬프를 이름으로 가진 디렉토리에 모델이 저장됨

!dir saved_models/

스위치가 틀립니다 - "".


In [ ]:
# 저장된 모델로부터 새로운 모델 로드 

new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.summary()

In [ ]:
# 복원된 모델 실행 

# 이 모델을 평가하려면 그전에 컴파일해야 합니다.
# 단지 저장된 모델의 배포라면 이 단계가 필요하지 않습니다.

new_model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))